In [37]:
import pandas as pd
from apyori import apriori

In [38]:
#encoding = "cp1252", encoding='utf-8', encoding='latin1'etc
base= pd.read_csv('../bases/MercadoSim.csv', sep=';',header=None)
base

,0,1,2,3,4,5,6
0,NaN,Cafe,NaN,Pao,Manteiga,NaN,NaN
1,Leite,NaN,Cerveja,Pao,Manteiga,NaN,NaN
2,NaN,Cafe,NaN,Pao,Manteiga,NaN,NaN
3,Leite,Cafe,NaN,Pao,Manteiga,NaN,NaN
4,NaN,NaN,Cerveja,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,Manteiga,NaN,NaN
6,NaN,NaN,NaN,Pao,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,Feijao
8,NaN,NaN,NaN,NaN,NaN,Arroz,Feijao
9,NaN,NaN,NaN,NaN,NaN,Arroz,NaN


In [39]:
base.shape

(10, 7)

**Como estaremos treinando um modelo a priori, que recebe entradas em formato de lista, precisamos transformar o dataframe em uma lista de transações.**

In [40]:
transacoes = [] 
for i in range(len(base)):
    transacoes.append([ str(base.values[i, j]) for j in range(base.shape[1]) if not pd.isna(base.values[i, j])])

In [41]:
transacoes

[['Cafe', 'Pao', 'Manteiga'],
 ['Leite', 'Cerveja', 'Pao', 'Manteiga'],
 ['Cafe', 'Pao', 'Manteiga'],
 ['Leite', 'Cafe', 'Pao', 'Manteiga'],
 ['Cerveja'],
 ['Manteiga'],
 ['Pao'],
 ['Feijao'],
 ['Arroz', 'Feijao'],
 ['Arroz']]

In [42]:
todos_itens = set()
for transacao in transacoes:
    for item in transacao:
        todos_itens.add(item)
todos_itens = list(todos_itens)

In [43]:
todos_itens

['Cafe', 'Arroz', 'Feijao', 'Cerveja', 'Manteiga', 'Pao', 'Leite']

In [44]:
transacoes_expandidas = []
for transacao in transacoes:
    transacao_expandida = transacao.copy()
    for item in todos_itens:
        if item not in transacao:
            transacao_expandida.append(f"NÃO_{item}")
    transacoes_expandidas.append(transacao_expandida)


In [45]:
transacao_expandida

['Arroz',
 'NÃO_Cafe',
 'NÃO_Feijao',
 'NÃO_Cerveja',
 'NÃO_Manteiga',
 'NÃO_Pao',
 'NÃO_Leite']

**Vamos chamar executar o algoritmo apriori e armazenar as regras obtidas**

In [46]:
regras = apriori(transacoes_expandidas, min_support=0.3, min_confidence=0.8)
saida = list(regras)
len(saida)

117

**Para facilitar a visualização, veja como transformar isso em um dataframe** 

Extraído de:

https://www.section.io/engineering-education/apriori-algorithm-in-python/

https://splunktool.com/understanding-apyoris-output

In [47]:
itemsets_com_suporte = []
for resultado in saida:
    itemset = list(resultado[0]) 
    suporte = resultado[1]        
    itemsets_com_suporte.append((itemset, suporte))

df_itemsets = pd.DataFrame(itemsets_com_suporte, columns=['Itemset', 'Suporte'])

#df_itemsets = df_itemsets.sort_values(by='Suporte', ascending=False)

print("Itemsets gerados com seus respectivos suportes:")
print(df_itemsets)

Itemsets gerados com seus respectivos suportes:
                                               Itemset  Suporte
0                                          [NÃO_Arroz]      0.8
1                                        [NÃO_Cerveja]      0.8
2                                         [NÃO_Feijao]      0.8
3                                          [NÃO_Leite]      0.8
4                                     [Cafe, Manteiga]      0.3
..                                                 ...      ...
112  [NÃO_Leite, NÃO_Cerveja, Manteiga, NÃO_Feijao,...      0.3
113  [NÃO_Cerveja, Manteiga, NÃO_Feijao, NÃO_Arroz,...      0.3
114  [NÃO_Leite, NÃO_Cerveja, NÃO_Feijao, NÃO_Arroz...      0.3
115  [NÃO_Leite, NÃO_Cerveja, NÃO_Cafe, NÃO_Pao, NÃ...      0.3
116  [NÃO_Cerveja, Cafe, Manteiga, NÃO_Feijao, NÃO_...      0.3

[117 rows x 2 columns]


In [48]:
df_itemsets

,Itemset,Suporte
0,[NÃO_Arroz],0.8
1,[NÃO_Cerveja],0.8
2,[NÃO_Feijao],0.8
3,[NÃO_Leite],0.8
4,"[Cafe, Manteiga]",0.3
...,...,...
112,"[NÃO_Leite, NÃO_Cerveja, Manteiga, NÃO_Feijao,...",0.3
113,"[NÃO_Cerveja, Manteiga, NÃO_Feijao, NÃO_Arroz,...",0.3
114,"[NÃO_Leite, NÃO_Cerveja, NÃO_Feijao, NÃO_Arroz...",0.3
115,"[NÃO_Leite, NÃO_Cerveja, NÃO_Cafe, NÃO_Pao, NÃ...",0.3


In [49]:
lhs = []
rhs = []
suporte = []
confianca = []
lift = []

for resultado in saida:
    s = resultado[1]
    result_rules = resultado[2]
    for result_rule in result_rules:
        a = list(result_rule[0])
        b = list(result_rule[1])
        c = result_rule[2]
        l = result_rule[3]
        
        # Filtrar para incluir apenas regras que envolvem ausência de itens
        if any(item.startswith("NÃO_") for item in a + b):
            lhs.append(a)
            rhs.append(b)
            suporte.append(s)
            confianca.append(c)
            lift.append(l)

# Criar DataFrame com as regras
RegrasFinais = pd.DataFrame({
    'Lhs': lhs,
    'Rhs': rhs,
    'suporte': suporte,
    'confianca': confianca,
    'lift': lift
})

RegrasFinais = RegrasFinais[
    (RegrasFinais['lift'] != 1.0) &  # Remove lift neutro
    (RegrasFinais['Lhs'].apply(len) > 1)  # Mantém apenas regras com combinações de itens
]

In [50]:
RegrasFinais

,Lhs,Rhs,suporte,confianca,lift
27,"[Cafe, Manteiga]",[NÃO_Arroz],0.3,1.0,1.250000
28,"[Cafe, NÃO_Arroz]",[Manteiga],0.3,1.0,2.000000
30,"[Cafe, Manteiga]",[NÃO_Cerveja],0.3,1.0,1.250000
31,"[Cafe, NÃO_Cerveja]",[Manteiga],0.3,1.0,2.000000
33,"[Cafe, Manteiga]",[NÃO_Feijao],0.3,1.0,1.250000
...,...,...,...,...,...
437,"[NÃO_Cerveja, Cafe, Manteiga, NÃO_Arroz, Pao]",[NÃO_Feijao],0.3,1.0,1.250000
438,"[Cafe, Manteiga, NÃO_Feijao, NÃO_Arroz, Pao]",[NÃO_Cerveja],0.3,1.0,1.250000
439,"[NÃO_Cerveja, Cafe, Manteiga, NÃO_Feijao, Pao]",[NÃO_Arroz],0.3,1.0,1.250000
440,"[NÃO_Cerveja, Cafe, NÃO_Feijao, NÃO_Arroz, Pao]",[Manteiga],0.3,1.0,2.000000


**Podemos ordenar estas regras por uma métrica desejada**

In [51]:
RegrasFinais.sort_values(by='lift', ascending =False)

,Lhs,Rhs,suporte,confianca,lift
441,"[NÃO_Cerveja, Manteiga, NÃO_Feijao, NÃO_Arroz,...",[Cafe],0.3,1.000000,3.333333
325,"[Cafe, NÃO_Arroz]","[Manteiga, Pao, NÃO_Cerveja]",0.3,1.000000,3.333333
339,"[Manteiga, Pao, NÃO_Arroz, NÃO_Cerveja]",[Cafe],0.3,1.000000,3.333333
423,"[Manteiga, Pao, NÃO_Cerveja]","[Cafe, NÃO_Feijao, NÃO_Arroz]",0.3,1.000000,3.333333
176,"[Manteiga, Pao, NÃO_Cerveja]",[Cafe],0.3,1.000000,3.333333
...,...,...,...,...,...
122,"[NÃO_Feijao, NÃO_Leite]",[NÃO_Cerveja],0.5,0.833333,1.041667
87,"[NÃO_Leite, NÃO_Arroz]",[NÃO_Feijao],0.5,0.833333,1.041667
88,"[NÃO_Feijao, NÃO_Leite]",[NÃO_Arroz],0.5,0.833333,1.041667
96,"[NÃO_Leite, NÃO_Cafe]",[NÃO_Cerveja],0.5,0.833333,1.041667
